In [238]:
# Modules
import os
import csv
import pandas as pd
import numpy as np

In [239]:
# Set path for files
schools_path = os.path.join("Resources", "schools_complete.csv")
print(schools_path)
students_path = os.path.join("Resources", "students_complete.csv")
print(students_path)


Resources\schools_complete.csv
Resources\students_complete.csv


In [240]:
# Use Pandas to read data
schools_table = pd.read_csv(schools_path)
schools_table.head()


,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [241]:
students_table = pd.read_csv(students_path)
students_table.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [242]:
total_schools = schools_table["school_name"].count()

total_student = students_table["Student ID"].count()

total_budget = schools_table["budget"].sum()

average_math_score = students_table["math_score"].mean()

average_reading_score = students_table["reading_score"].mean()

overall_score = (average_math_score+average_reading_score)/2


In [243]:
schools_table_pd = pd.DataFrame(schools_table)
students_table_pd = pd.DataFrame(students_table)


In [244]:
math_pass_rate = students_table_pd.loc[students_table_pd["math_score"] > 69, ["math_score"]].count()/total_student

reading_pass = students_table.loc[students_table["reading_score"] > 69, ["math_score"]].count()/total_student


In [245]:
summary_pd_1 = pd.DataFrame({"Total Schools" : [total_schools] ,
                             "Total Students": [total_student],
                             "Total Budget" : [total_budget],
                             "Average Math Score" : [average_math_score] ,
                             "Average Reading Score" : [average_reading_score] ,
                             "% Passing Math" : [math_pass_rate] ,
                             "% Passing Reading" : [reading_pass] ,
                             "% Overall Passing Rate" :[total_schools]
                            })
summary_pd_1

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,math_score 0.749809 dtype: float64,math_score 0.858055 dtype: float64,15


In [131]:
merged_table = pd.merge(students_table_pd, schools_table_pd, on="school_name")
merged_table.head()


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [132]:
schools_table_pd1 = schools_table_pd.set_index("school_name")
schools_table_pd1.head()

,School ID,type,size,budget
school_name,,,,
Huang High School,0,District,2917,1910635
Figueroa High School,1,District,2949,1884411
Shelton High School,2,Charter,1761,1056600
Hernandez High School,3,District,4635,3022020
Griffin High School,4,Charter,1468,917500


In [133]:
grouped_merged_table = merged_table.groupby(['school_name'])
# find average math and reading score by using mean on grouped table
grouped_merged_table_mean= grouped_merged_table[["reading_score","math_score"]].mean()

In [134]:
average_budget_per_student = schools_table_pd1[["budget"]]/grouped_merged_table[["budget"]].count()

math_pass_per_school = merged_table.loc[merged_table["math_score"] > 69, ["school_name","math_score"]]
grouped_math_pass_per_school = math_pass_per_school.groupby(['school_name'])
math_rate_pass=  grouped_math_pass_per_school.count()/grouped_merged_table[["math_score"]].count()

reading_pass_per_school = merged_table.loc[merged_table["reading_score"] > 69, ["school_name","reading_score"]]
grouped_reading_pass_per_school = reading_pass_per_school.groupby(['school_name'])
reading_rate_pass=  grouped_reading_pass_per_school.count()/grouped_merged_table[["reading_score"]].count()

total_rate_pass = (math_rate_pass["math_score"] + reading_rate_pass["reading_score"])/2


In [135]:
summary_school = pd.merge(schools_table_pd1, grouped_merged_table_mean, on="school_name")
summary_school["% Passing Math"] = math_rate_pass *100
summary_school["% Passing Reading"] = reading_rate_pass*100
summary_school["% Overall Passing Rate"] = total_rate_pass*100
summary_school["Per Student Budget"] = average_budget_per_student
del summary_school['School ID']
summary_school = summary_school.rename(columns={"size" : "Total Students",
                                                "budget": "Total School Budget",
                                                "math_score": "Average Math Score",
                                                "reading_score":"Average Reading Score"}) 
final_summary_school = summary_school[["type",
                                       "Total Students",
                                       "Total School Budget",
                                       "Per Student Budget",
                                       "Average Math Score",
                                       "Average Reading Score",
                                      "% Passing Math",
                                      "% Passing Reading",
                                      "% Overall Passing Rate"]]
final_summary_school.head()

,type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,93.867121,95.854628,94.860875
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,73.807983
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668


In [136]:
# Top Performing Schools (By Passing Rate)
final_summary_school_by_pass_rate = final_summary_school.sort_values(["% Overall Passing Rate"], ascending=False)
final_summary_school_by_pass_rate.head()

,type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,95.203679


In [137]:
# Bottom Performing Schools (By Passing Rate)
final_summary_school_by_low_pass_rate = final_summary_school.sort_values(["% Overall Passing Rate"])
final_summary_school_by_low_pass_rate.head()

,type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308


In [138]:
merged_table.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [139]:
schools_table_pd1["Reading score 9th"] = merged_table.loc[merged_table["grade"] == "9th", 
                                        ["school_name","grade","reading_score"]].groupby(['school_name']).mean()
schools_table_pd1["Reading score 10th"] = merged_table.loc[merged_table["grade"] == "10th", 
                                        ["school_name","grade","reading_score"]].groupby(['school_name']).mean()
schools_table_pd1["Reading score 11th"] = merged_table.loc[merged_table["grade"] == "11th", 
                                        ["school_name","grade","reading_score"]].groupby(['school_name']).mean()
schools_table_pd1["Reading score 12th"] = merged_table.loc[merged_table["grade"] == "12th", 
                                        ["school_name","grade","reading_score"]].groupby(['school_name']).mean()

schools_table_pd1["Math score 9th"] = merged_table.loc[merged_table["grade"] == "9th", 
                                        ["school_name","grade","math_score"]].groupby(['school_name']).mean()
schools_table_pd1["Math score 10th"] = merged_table.loc[merged_table["grade"] == "10th", 
                                        ["school_name","grade","math_score"]].groupby(['school_name']).mean()
schools_table_pd1["Math score 11th"] = merged_table.loc[merged_table["grade"] == "11th", 
                                        ["school_name","grade","math_score"]].groupby(['school_name']).mean()
schools_table_pd1["Math score 12th"] = merged_table.loc[merged_table["grade"] == "12th", 
                                        ["school_name","grade","math_score"]].groupby(['school_name']).mean()

In [140]:
# Math Scores by Grade
schools_table_pd1[["Math score 9th","Math score 10th","Math score 11th","Math score 12th"]]


,Math score 9th,Math score 10th,Math score 11th,Math score 12th
school_name,,,,
Huang High School,77.027251,75.908735,76.446602,77.225641
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Shelton High School,83.420755,82.917411,83.383495,83.778976
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Griffin High School,82.044010,84.229064,83.842105,83.356164
Wilson High School,83.085578,83.724422,83.195326,83.035794
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Bailey High School,77.083676,76.996772,77.515588,76.492218
Holden High School,83.787402,83.429825,85.000000,82.855422


In [141]:
# reading Scores by Grade
schools_table_pd1[["Reading score 9th","Reading score 10th","Reading score 11th","Reading score 12th"]]

,Reading score 9th,Reading score 10th,Reading score 11th,Reading score 12th
school_name,,,,
Huang High School,81.290284,81.512386,81.417476,80.305983
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Shelton High School,84.122642,83.441964,84.373786,82.781671
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Griffin High School,83.369193,83.706897,84.288089,84.013699
Wilson High School,83.939778,84.021452,83.764608,84.317673
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Bailey High School,81.303155,80.907183,80.945643,80.912451
Holden High School,83.677165,83.324561,83.815534,84.698795


In [163]:
# create bins
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]
Spending_Ranges  = pd.cut(final_summary_school["Per Student Budget"], spending_bins, labels=group_names)
Spending_Ranges_df=pd.DataFrame(Spending_Ranges)
merged_table_with_ranges = pd.merge(merged_table, Spending_Ranges_df, on="school_name")
merged_table_with_ranges.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget,Per Student Budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635,$645-675
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635,$645-675
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635,$645-675
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635,$645-675
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635,$645-675


In [201]:
merged_table_with_ranges_grouped = merged_table_with_ranges.groupby(['Per Student Budget'])
merged_table_with_ranges_grouped_mean = merged_table_with_ranges_grouped[["reading_score","math_score"]].mean()

math_pass_per_school1 = merged_table_with_ranges.loc[merged_table_with_ranges["math_score"] > 69,
                                                     ["Per Student Budget","math_score"]].groupby(['Per Student Budget']).count()
reading_pass_per_school1 = merged_table_with_ranges.loc[merged_table_with_ranges["reading_score"] > 69,
                                                     ["Per Student Budget","reading_score"]].groupby(['Per Student Budget']).count()
math_pass_rate = math_pass_per_school1 / merged_table_with_ranges_grouped[["math_score"]].count()
reading_pass_rate =  reading_pass_per_school1 / merged_table_with_ranges_grouped[["reading_score"]].count()
total_rate = pd.DataFrame((math_pass_rate["math_score"] + reading_pass_rate["reading_score"]) /2)
merged_table_with_ranges_grouped_mean["% Passing Math"] = math_pass_rate *100
merged_table_with_ranges_grouped_mean["% Passing Reading"] = reading_pass_rate *100
merged_table_with_ranges_grouped_mean["% Overall Passing Rate"] = total_rate *100
merged_table_with_ranges_grouped_mean

,reading_score,math_score,% Passing Math,% Passing Reading,% Overall Passing Rate
Per Student Budget,,,,,
<$585,83.964039,83.363065,93.702889,96.686558,95.194724
$585-615,83.838414,83.529196,94.124128,95.886889,95.005509
$615-645,81.434088,78.061635,71.400428,83.614770,77.507599
$645-675,81.005604,77.049297,66.230813,81.109397,73.670105


In [209]:
merged_table.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [215]:
# size_bins
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
size_Ranges  = pd.cut(merged_table["size"], size_bins, labels=group_names)
merged_table["School Size"] = pd.DataFrame(size_Ranges)
merged_table.head()


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget,School Size
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635,Large (2000-5000)
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635,Large (2000-5000)
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635,Large (2000-5000)
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635,Large (2000-5000)
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635,Large (2000-5000)


In [221]:
merged_table_school_size_grouped = merged_table.groupby(['School Size'])
merged_table_school_size_grouped_mean = merged_table_school_size_grouped[["reading_score","math_score"]].mean()

math_pass_per_school2 = merged_table.loc[merged_table["math_score"] > 69,
                                                     ["School Size","math_score"]].groupby(['School Size']).count()
reading_pass_per_school2 = merged_table.loc[merged_table["reading_score"] > 69,
                                                     ["School Size","reading_score"]].groupby(['School Size']).count()
math_pass_rate2 = math_pass_per_school2 / merged_table_school_size_grouped[["math_score"]].count()
reading_pass_rate2 =  reading_pass_per_school2 / merged_table_school_size_grouped[["reading_score"]].count()
total_rate2 = pd.DataFrame((math_pass_rate2["math_score"] + reading_pass_rate2["reading_score"]) /2)
merged_table_school_size_grouped_mean["% Passing Math"] = math_pass_rate2 *100
merged_table_school_size_grouped_mean["% Passing Reading"] = reading_pass_rate2 *100
merged_table_school_size_grouped_mean["% Overall Passing Rate"] = total_rate2 *100
merged_table_school_size_grouped_mean

,reading_score,math_score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Large (2000-5000),81.198674,77.477597,68.652380,82.125158,75.388769
Medium (1000-2000),83.867989,83.372682,93.616522,96.773058,95.194790
Small (<1000),83.974082,83.828654,93.952484,96.040317,94.996400


In [222]:
merged_table_school_type_grouped = merged_table.groupby(['type'])
merged_table_school_type_grouped_mean = merged_table_school_type_grouped[["reading_score","math_score"]].mean()

math_pass_per_school3 = merged_table.loc[merged_table["math_score"] > 69,
                                                     ["type","math_score"]].groupby(['type']).count()
reading_pass_per_school3 = merged_table.loc[merged_table["reading_score"] > 69,
                                                     ["type","reading_score"]].groupby(['type']).count()
math_pass_rate3 = math_pass_per_school3 / merged_table_school_type_grouped[["math_score"]].count()
reading_pass_rate3 =  reading_pass_per_school3 / merged_table_school_type_grouped[["reading_score"]].count()
total_rate3 = pd.DataFrame((math_pass_rate3["math_score"] + reading_pass_rate3["reading_score"]) /2)
merged_table_school_type_grouped_mean["% Passing Math"] = math_pass_rate3 *100
merged_table_school_type_grouped_mean["% Passing Reading"] = reading_pass_rate3 *100
merged_table_school_type_grouped_mean["% Overall Passing Rate"] = total_rate3 *100
merged_table_school_type_grouped_mean

,reading_score,math_score,% Passing Math,% Passing Reading,% Overall Passing Rate
type,,,,,
Charter,83.902821,83.406183,93.701821,96.645891,95.173856
District,80.962485,76.987026,66.518387,80.905249,73.711818
